In [1]:
import numpy as np
from scipy.optimize import minimize
import sympy
from tqdm import tqdm

from environments import ARESEAOptimization

In [2]:
senv = ARESEAOptimization(backend="simulation", backendargs={"measure_beam": "direct"})
menv = ARESEAOptimization(backend="machine", backendargs={"measure_beam": "us"})

## Screen Misalignment
Turn everything off. Assuming we are in the golden orbit, the position on the screen gives the screen's misalignment.

In [4]:
menv.next_initial = np.zeros(5)
menv.reset()
beam_parameters = menv.backend.compute_beam_parameters()
screen_misalignment = (beam_parameters[0], beam_parameters[1])
screen_misalignment

(-0.0009826608, 0.000195752)

## Run Quadrupole Scan
This can be used to get the quadrupole misalignments. The scan of AREAMQZM1 is also used to characterise the incoming beam.

In [5]:
menv.next_initial = "stay"
menv.reset()

quadnames = ["AREAMQZM1", "AREAMQZM2", "AREAMQZM3"]
quadindices = [0, 1, 3]
quadvalues = [
    np.linspace(-6, 6, num=9),
    np.linspace(-6, 6, num=3),
    np.linspace(-6, 6, num=3)
]

beam_parameters = []
for quadrupole, i, k1s in zip(quadnames, quadindices, quadvalues):
    quad_beam_parameters = []
    for k1 in tqdm(k1s):
        actuators = np.zeros(5)
        actuators[i] = k1
        quad_beam_parameters.append(menv.backend.compute_beam_parameters())
    beam_parameters.append(quad_beam_parameters)

100%|██████████| 3/3 [00:13<00:00,  4.58s/it]


## Find Quadrupole Misalignments
Optimise for the misalignments of the quadrupoles using the data from the scan above.

In [ ]:
quadrupole_misalignments = []
for i, (quadindex, k1s, quad_beam_parameters) in enumerate(zip(quadindices, quadvalues, beam_parameters)):
    def optfn(misalignment):
        senv.backend.next_misalignments = np.zeros(8)
        senv.backend.next_misalignments[-2:] = screen_misalignment
        senv.backend.next_misalignments[2*i:2*i+2] = misalignment

        senv.backend.next_incoming = {"mu_x": 0, "mu_xp": 0, "mu_y": 0, "mu_yp": 0}

        senv.reset()
        actuators = np.zeros(5)
        actuators[quadindex] = k1s[0]
        senv.backend.actuators = actuators
        beampos_simulation = np.zeros(4)
        beampos_simulation[:2] = senv.backend.compute_beam_parameters()[:2]
        actuators = np.zeros(5)
        actuators[quadindex] = k1s[-1]
        senv.backend.actuators = actuators
        beampos_simulation[2:] = senv.backend.compute_beam_parameters()[:2]

        beampos_machine = np.array([
            quad_beam_parameters[0][0],
            quad_beam_parameters[0][1],
            quad_beam_parameters[-1][0],
            quad_beam_parameters[-1][1]
        ])

        return np.mean((beampos_machine - beampos_simulation)**2)
    
    optresult = minimize(optfn, x0=(0,0), method="Nelder-Mead")
    quadrupole_misalignments.append(optresult["x"])

quadrupole_misalignments

In [13]:
quadrupole_misalignments = []
for i, (quadindex, k1s, quad_beam_parameters) in enumerate(zip(quadindices, quadvalues, beam_parameters)):
    def optfn(misalignment):
        senv.backend.next_misalignments = np.zeros(8)
        senv.backend.next_misalignments[-2:] = screen_misalignment
        senv.backend.next_misalignments[2*i:2*i+2] = misalignment

        senv.backend.next_incoming = {"mu_x": 0, "mu_xp": 0, "mu_y": 0, "mu_yp": 0}

        senv.reset()
        actuators = np.zeros(5)
        actuators[quadindex] = k1s[0]
        senv.backend.actuators = actuators
        beampos_simulation = np.zeros(4)
        beampos_simulation[:2] = senv.backend.compute_beam_parameters()[:2]
        actuators = np.zeros(5)
        actuators[quadindex] = k1s[-1]
        senv.backend.actuators = actuators
        beampos_simulation[2:] = senv.backend.compute_beam_parameters()[:2]

        beampos_machine = np.array([
            quad_beam_parameters[0][0],
            quad_beam_parameters[0][1],
            quad_beam_parameters[-1][0],
            quad_beam_parameters[-1][1]
        ])

        return np.mean((beampos_machine - beampos_simulation)**2)
    
    optresult = minimize(optfn, x0=(0,0), method="Nelder-Mead")
    quadrupole_misalignments.append(optresult["x"])

quadrupole_misalignments

[array([0., 0.]), array([0., 0.]), array([0., 0.])]

## Infer Missing Beam Parameters
Assuming the the beam is on the golden orbit and now knowing more about the misalignments, we can optimise for the remaining parameters of the incoming beam.

In [14]:
def optfn(parameters):
    senv.backend.next_incoming = {
        "mu_x": 0,
        "mu_xp": 0,
        "mu_y": 0,
        "mu_yp": 0,
        "sigma_x": parameters[0],
        "sigma_xp": parameters[1],
        "sigma_y": parameters[2],
        "sigma_yp": parameters[3]
    }
    senv.backend.misalignments = [
        quadrupole_misalignments[0][0],
        quadrupole_misalignments[0][1],
        quadrupole_misalignments[1][0],
        quadrupole_misalignments[1][1],
        quadrupole_misalignments[2][0],
        quadrupole_misalignments[2][1],
        screen_misalignment[0],
        screen_misalignment[1]
    ]

    senv.reset()


    for k1 in quadvalues[0]:
        senv.backend.actuators = [k1, 0, 0, 0, 0]
        


In [68]:
mu0 = sympy.MatrixSymbol("mu_0", 7, 1)
mu1 = sympy.MatrixSymbol("mu_1", 7, 1)
R = sympy.MatrixSymbol("R", 7, 7)
mueq = sympy.Eq(mu1, sympy.MatMul(R, mu0), evaluate=False)
mueq

Eq(mu_1, R*mu_0)

In [75]:
mueq = sympy.Eq(sympy.MatMul(R**(-1), mu1), sympy.MatMul(R**(-1), R, mu0), evaluate=False)
mueq

Eq(R**(-1)*mu_1, R**(-1)*R*mu_0)

In [71]:
cov0 = sympy.MatrixSymbol("C_0", 7, 7)
cov1 = sympy.MatrixSymbol("C_1", 7, 7)
R = sympy.MatrixSymbol("R", 7, 7)
coveq = sympy.Eq(cov1, sympy.MatMul(R, cov0, R.T), evaluate=False)
coveq

Eq(C_1, R*C_0*R.T)

In [73]:
coveq = sympy.Eq(sympy.MatMul(R**(-1), cov1, R.T**(-1)), sympy.MatMul(R**(-1), R, cov0, R.T, R.T**(-1)), evaluate=False)
coveq

Eq(R**(-1)*C_1*R.T**(-1), R**(-1)*R*C_0*R.T*R.T**(-1))